### Requirements

- langchain>=0.0.200
- openai>=0.27.8
- chromadb>=0.4.7
- beautifulsoup4>=4.12.2
- selenium>=4.11.2
- webdriver-manager>=3.8.6
- python-dotenv>=1.0.0
- pprintpp>=0.4.0

## 트러블 슈팅

> 2024/11/22 (성진)
---
**데이터 전처리 과정**
1. WebBaseLoader 로 레퍼런스가 정상적으로 인식되지 않는 문제  
 - 원인 : WebBaseLoader의 한계로 인한 실패 가능성. WebBaseLoader는 간단한 텍스트 추출에 적합하며, 복잡한 HTML 구조나 JavaScript 기반의 동적 콘텐츠를 처리하지 못할 수 있다. 

    1_1. WebBaseLoader 대신 request를 사용한 HTML 크롤링 시도 -> 실패  
     - 원인 : Notion 페이지는 JavaScript를 활용한 동적 렌더링을 사용하는 구조여서 requests만 사용해서는 JavaScript로 렌더링된 내용을 가져올 수 없었다.

    1_2. Selenium 사용 --> 부분 성공  
     - 원인 : implicitly_wait 으로 대기 후 로드 시 데이터를 가져오는 경우가 있고, 못가져오는 경우가 있었음..

    1_3. 대기 방식 변경 (WebDriverWait.until() 사용) --> 성공  
     - 로드하려는 웹페이지의 html 중 특정 요소가 로드될 때까지 wait하는 방식 사용

2. soup.get_text() 에서 노션 문서의 헤드라인만 추출되는 문제  
 - 원인 : 노션 페이지에 진입했을 때, 토글(Ctrl+Alt+T)이 닫힌 상태로 텍스트가 추출되고 있었음

    2_1. 단축키 입력을 선언하는 방법 (헤드리스 상태에서 사용 불가)  

    2_2. html 내에 토글 버튼을 찾아 여는 방법 (헤드리스 상태에서 사용 가능) --> 선택 (성공)  
     - drive.find_elements 로 버튼 탐색 후 클릭 (지연시간 1초)

3. 추출된 텍스트 파일에 일부 문법 구문이 남아있는 문제  
 - re (regular expression) 을 활용해 삭제/전처리 (성공)

**유사한 질문을 계속해서 질문하는 경우**
1. cosine 유사도 비교로 유사한 질문을 생성해내는 경우에는 다른 질문을 생성하도록 했음
   - cosine 유사도는 단어의 유사도를 비교하는 것에 약점을 보였기 때문에 성능이 크게 좋아지지 않음 (실패)
   - 프롬프팅으로 고도화 시도 (성공))


---
**AI 챗봇**

1. AI 가 질문에 대한 사용자의 답변을 인식하지 못하는 문제  
 - 사용자의 답변을 질문과 함께 feedback_prompt 형식으로 묶어서 다시 AI에게 전달하는 방식으로 문제 해결

2. AI 가 동일한 주제에 대해서 반복적으로 비슷한 질문만을 하는 문제 --> (해결중..)

---

> 2024/11/26 (수연)

2. AI 가 동일한 주제에 대해서 반복적으로 비슷한 질문만을 하는 문제 -> 해결

   - 해결 방법 : 
   - 수동적으로 참고할 docs와 갯수를 할당 (코드에서는 한 교재당 2개의 질문)
   - 유사한 질문을 rag에 넘겨주어 겹치지 않도록 프롬프팅 (직전 5개의 문장 기준, 이미 한 docs당 2개의 문장 질문 제한 고려)
   - 프롬프팅 고도화


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from dotenv import load_dotenv
import os

from selenium.webdriver.common.keys import Keys

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
def save_file(txt:str, file_name:str):

    with open(file_name, 'w', encoding='utf-8') as content_file:
        content_file.write(txt)

    print(f"TEXT 파일 저장 완료: {file_name}")

In [2]:

# .env 파일에서 환경변수 로드
load_dotenv()
api_key = os.getenv("OPEN_AI_KEY")

llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

### 교재 저장하는 부분

In [ ]:
    save_file(''.join(txt), f"DL_dataset_{j}.txt")
    


driver.quit()  # 브라우저 종료


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

4
TEXT 파일 저장 완료: DL_dataset_1.txt
6
TEXT 파일 저장 완료: DL_dataset_2.txt
5
TEXT 파일 저장 완료: DL_dataset_3.txt
3
TEXT 파일 저장 완료: DL_dataset_4.txt
5
TEXT 파일 저장 완료: DL_dataset_5.txt
4
TEXT 파일 저장 완료: DL_dataset_6.txt
3
TEXT 파일 저장 완료: DL_dataset_7.txt
4
TEXT 파일 저장 완료: DL_dataset_8.txt
3
TEXT 파일 저장 완료: DL_dataset_9.txt
3
TEXT 파일 저장 완료: DL_dataset_10.txt
3
TEXT 파일 저장 완료: DL_dataset_11.txt
4
TEXT 파일 저장 완료: DL_dataset_12.txt
2
TEXT 파일 저장 완료: DL_dataset_13.txt
4
TEXT 파일 저장 완료: DL_dataset_14.txt
2
TEXT 파일 저장 완료: DL_dataset_15.txt
3
TEXT 파일 저장 완료: DL_dataset_16.txt


### 저장한 교재 (txt) 불러오는 코드

In [ ]:
import os
text_list = []

# 파일을 읽어와서 text_list에 저장하는 함수
def load_files_to_list(file_path, text_list):
    if os.path.exists(file_path):  # 파일이 존재하는지 확인
        with open(file_path, 'r', encoding='utf-8') as file:  # 파일 열기
            content = file.read()  # 파일 내용 읽기
            text_list.append(content)  # text_list에 추가
    else:
        print(f"File {file_path} not found.")
        
    

# 예시: 파일 경로 목록을 지정
file_paths = [f"DL_dataset_{i}.txt" for i in range(1, 17)]  # 파일 경로 목록 (DL_dataset_1.txt, DL_dataset_2.txt, ...)

print(file_paths)

for j in range(len(file_paths)):
    load_files_to_list(file_paths[j], text_list)

text_list


### 각 교재별 retriever 생성부분

In [4]:
from langchain.schema import Document

# text_list를 Document 객체로 변환
documents = [Document(page_content=text) for text in text_list]


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

def get_retriever(texts:str):

    # text_list를 Document 객체로 변환
    documents = [Document(page_content=texts)]

    recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 200,
    chunk_overlap= 20,
    length_function=len,
    is_separator_regex=False,
    )

    splits_recur = recursive_text_splitter.split_documents(documents)
    splits = splits_recur

    print("Top 10 chunks:")
    for i, chunk in enumerate(splits[:10], 1):
        pprint(f"\nChunk {i}:\n{chunk.page_content}")
    # OpenAI 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
    bm25_retriever = BM25Retriever.from_documents(splits)
    faiss_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    retriever = EnsembleRetriever(
                retrievers=[bm25_retriever, faiss_retriever],
                weights=[0.5, 0.5]  # 가중치 설정 (가중치의 합은 1.0)
            )

    return retriever

In [6]:
retriever_list = []

docs_1_retriever = get_retriever(text_list[0])
retriever_list.append(docs_1_retriever)
docs_2_retriever = get_retriever(text_list[1])
retriever_list.append(docs_2_retriever)
docs_3_retriever = get_retriever(text_list[2])
retriever_list.append(docs_3_retriever)
docs_4_retriever = get_retriever(text_list[3])
retriever_list.append(docs_4_retriever)
docs_5_retriever = get_retriever(text_list[4])
retriever_list.append(docs_5_retriever)
docs_6_retriever = get_retriever(text_list[5])
retriever_list.append(docs_6_retriever)
docs_7_retriever = get_retriever(text_list[6])
retriever_list.append(docs_7_retriever)
docs_8_retriever = get_retriever(text_list[7])
retriever_list.append(docs_8_retriever)
docs_9_retriever = get_retriever(text_list[8])
retriever_list.append(docs_9_retriever)
docs_10_retriever = get_retriever(text_list[9])
retriever_list.append(docs_10_retriever)
docs_11_retriever = get_retriever(text_list[10])
retriever_list.append(docs_11_retriever)
docs_12_retriever = get_retriever(text_list[11])
retriever_list.append(docs_12_retriever)
docs_13_retriever = get_retriever(text_list[12])
retriever_list.append(docs_13_retriever)
docs_14_retriever = get_retriever(text_list[13])
retriever_list.append(docs_14_retriever)
docs_15_retriever = get_retriever(text_list[14])
retriever_list.append(docs_15_retriever)

Top 10 chunks:
('\n'
 'Chunk 1:\n'
 '[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다!📘[SCC] 기초가 탄탄한 딥러닝/📚[스파르타코딩클럽] 기초가 탄탄한 딥러닝 - '
 '1주차/📕[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다!Made with📕[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다![수업 '
 '목표]딥러닝이 무엇인지 개념에 대해 알아봅시다.딥러닝의 역사와 어디에 사용할 수 있을지')
('\n'
 'Chunk 2:\n'
 '역사와 어디에 사용할 수 있을지 알아봅시다[목차]01. 딥러닝이란 무엇일까요?02. 딥러닝의 역사와 활용 방안03. 딥러닝을 배워야 하는 '
 '이유💡모든 토글을 열고 닫는 단축키')
'\nChunk 3:\nWindows : Ctrl + alt + t'
('\n'
 'Chunk 4:\n'
 'Mac : ⌘ + ⌥ + t 01. 딥러닝이란 무엇일까요?✔️딥러닝이란 무엇인지 개념에 대해서 알아봅시다!1) 딥러닝이란?☑️ 딥러닝 '
 '개념딥러닝은 인공신경망(Artificial Neural Networks)을 기반으로 한 기계 학습의 한 분야입니다.다층 신경망을 사용하여 '
 '데이터로부터 특징을 자동으로 학습하고, 이를 통해 복잡한 문제를 해결합니다.입력')
('\n'
 'Chunk 5:\n'
 '통해 복잡한 문제를 해결합니다.입력 데이터에서 중요한 패턴을 추출하고, 이를 바탕으로 예측, 분류, 생성 등의 다양한 작업을 수행할 수 '
 '있습니다.ALT☑️ 딥러닝의 특징비선형 추론: 딥러닝은 비선형 추론을 통해 복잡한 데이터의 패턴을 학습할 수 있습니다.다층 구조: 여러 '
 '층의 신경망을 사용하여 데이터의 고차원 특징을 학습합니다.자동 특징 추출:')
('\n'
 'Chunk 6:\n'
 '특징을 학습합니다.자동 특징 추출: 데이터로부터 중요한 특징을 자동으로 추출하여 별도의 특징 공학(feature engineering) '
 '과정이 필요 없습니다.02. 딥러닝의 역사와 활용 방안✔️딥러닝

In [7]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

In [8]:
# Prompt 및 Chain 구성
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = " ".join([doc.page_content for doc in inputs["context"]])
        print(f"Context output: {context_text}")
        return {"context": context_text, "quiz_list": inputs["quiz_list"]}

prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 딥러닝을 가르치는 AI 강사입니다.
    아래의 {context} 바탕으로만 하나의 질문과 답변 한 쌍을 생성해주세요.
    (최대한 코드에 관한 시나리오적 질문이면 더 좋습니다.)
    {quiz_list}에 존재하는 질문들과는 최대한 덜 유사한 질문을 생성해주세요.
    아래의 제약 조건과 출제 방식에 맞춘 질문을 생성해주세요.
     
    제약 조건:
    1. "Context"에서 제공된 내용만 기반으로 질문을 생성하세요.
    2. AI 관련 내용이 아닌 질문은 생성하지 마세요
    3. "QuizList"에 이미 있는 질문과 유사하지 않은 새로운 질문을 생성하세요.

    출제 방식:
    - 질문은 반드시 보기가 있는 객관식(MCQ) 또는 O,X 형태로 출제하세요.
    - "Context"에 명시적으로 언급된 개념, 정의, 또는 내용을 활용하세요.
    
    Context:
    {context}
    
    QuizList:
    {quiz_list}

    """)
])



In [9]:
# RAG Chain 생성 함수
def create_rag_chain(retriever):
    return (
        {
            "context": retriever,
            "quiz_list": DebugPassThrough()
        }
        | DebugPassThrough()  # DebugPassThrough()가 실제로 어떤 역할을 하는지 확인
        | ContextToText()     # Text 변환을 위한 ContextToText
        | prompt              # prompt 사용
        | llm                 # LLM 호출
        | StrOutputParser()   # 출력 파서
    )


#### 트러블 슈팅 (cosine 유사도 기반 유사한 문장 제외 코드)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# from langchain_openai import OpenAIEmbeddings

# # 임베딩을 가져오는 함수
# def get_embeddings(text, api_key):
#     """텍스트를 임베딩 벡터로 변환"""
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
#     return embeddings.embed_documents([text])[0]  # 텍스트를 임베딩 벡터로 변환

# # 코사인 유사도 계산 함수
# def calculate_cosine_similarity(embedding1, embedding2):
#     """두 벡터 간의 코사인 유사도를 계산"""
#     return cosine_similarity([embedding1], [embedding2])[0][0]

# # 유사한 퀴즈가 있는지 확인하는 함수
# def is_similar_to_existing_quizzes(quiz, quiz_list, api_key, threshold=0.8):
#     """
#     주어진 퀴즈가 quiz_list에 있는 퀴즈들과 유사한지 비교하여
#     유사도가 threshold 이상이면 True를 반환하고, 그렇지 않으면 False를 반환.
#     """
#     quiz_embedding = get_embeddings(quiz, api_key)  # 새로 생성한 퀴즈의 임베딩 벡터
    
#     for existing_quiz in quiz_list:
#         existing_quiz_embedding = get_embeddings(existing_quiz, api_key)
#         similarity = calculate_cosine_similarity(quiz_embedding, existing_quiz_embedding)
        
#         if similarity > threshold:  # 유사도가 threshold 이상이면 유사한 퀴즈로 간주
#             return True
#     return False


### 동작부분

In [ ]:
import random
j = 0
cnt = 0
quiz_list = []

# 사용자가 "exit" 을 입력할 경우, 대화가 종료됩니다.

while True: 

    if (j %2 == 0 and j > 0):
        cnt += 1
        print(f"{cnt} 번째 docs 참고")
    print("퀴즈를 생성합니다 ===============")
    quiz_list = quiz_list[-5:]  # 최신 5개 퀴즈만 보관
    quiz_str = " ".join(quiz_list)
    rag_chain = create_rag_chain(retriever_list[cnt])

    query = prompt.format(
    context=retriever_list[cnt],  # 'context' 값 전달
    quiz_list=quiz_list,          # 'quiz_list' 값 전달
    )


    try:
        response = rag_chain.invoke("퀴즈 하나를 생성해줘")
    except Exception as e:
        print(f"Quiz 생성 중 오류 발생: {e}")
        break
    
    quiz = response
    
    quiz_list.append(quiz)
    print(f"Quiz : {quiz}")
    j+=1
    save_file(''.join(str(quiz)), f"quiz_list_{j}.txt")
    print(f"Updated quiz_list: {quiz_list}")
    
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요: ").strip()

    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    
    if not user_answer:
        print("답변이 비어 있습니다. 다시 입력해주세요.")
        continue

    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
        AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
        피드백은 아래와 같은 형식이어야 합니다:
        - 정답 여부: "N번" 또는 "예/아니오"
        - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
         
        퀴즈: {quiz}
        사용자의 답변: {user_answer}
        피드백:
        """)
    ])
    feedback_chain = feedback_prompt | llm
    feedback = feedback_chain.invoke({"quiz": quiz, "answer": user_answer})
    print("Feedback:")
    pprint(feedback)



퀴즈를 생성합니다 ===============
Debug Output: 퀴즈 하나를 생성해줘
Debug Output: {'context': [Document(metadata={}, page_content='CT 스캔 이미지를 분석하여 암을 조기에 발견할 수 있습니다.03. 딥러닝을 배워야 하는 이유✔️그러면 왜? 딥러닝을 배워야 하는지 이유를 알아 봅시다1) 딥러닝을 배워야 하는 이유'), Document(metadata={}, page_content='음성 인식딥러닝은 음성 인식 시스템의 성능을 크게 향상시켰습니다.예를 들어, 애플의 Siri, 아마존의 Alexa와 같은 가상 비서는 딥러닝을 사용하여 사용자의 음성을 인식하고 명령을 수행합니다.☑️ 의료 분야의료 영상 분석, 질병 예측, 신약 개발 등 다양한 의료 분야에서도 활용됩니다.예를 들어, 딥러닝 모델은 MRI나 CT 스캔 이미지를 분석하여 암을'), Document(metadata={}, page_content='[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다!📘[SCC] 기초가 탄탄한 딥러닝/📚[스파르타코딩클럽] 기초가 탄탄한 딥러닝 - 1주차/📕[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다!Made with📕[스파르타코딩클럽] 1. 딥러닝 개념을 잡아봅시다![수업 목표]딥러닝이 무엇인지 개념에 대해 알아봅시다.딥러닝의 역사와 어디에 사용할 수 있을지'), Document(metadata={}, page_content='역사와 어디에 사용할 수 있을지 알아봅시다[목차]01. 딥러닝이란 무엇일까요?02. 딥러닝의 역사와 활용 방안03. 딥러닝을 배워야 하는 이유💡모든 토글을 열고 닫는 단축키'), Document(metadata={}, page_content='이미지 생성 등 다양한 이미지 처리 작업에 활용됩니다. 예를 들어, 자율 주행 자동차는 딥러닝을 사용하여 도로 상황을 인식하고, 보행자와 차량을 감지합니다.☑️ 자연어 처리번역, 요약, 감정 분석 등 자연어 처리 작업에 사용됩니다. 예를 